In [3]:
import pandas as pd

import requests
from bs4 import BeautifulSoup
import re
import time



web scraping recurcive functions 

In [4]:
def get_article_details(url):
    try:
        data = requests.get(url).text
        soup = BeautifulSoup(data, "html.parser")
        
        title = soup.title.text if soup.title else "No Title"
        image = soup.find("div", attrs={"class": "img-cont"}).find("img")["src"] if soup.find("div", attrs={"class": "img-cont"}) else "No Image"
        writer = soup.find("span", attrs={"class": "writeBy"}).text.replace("كتبت", "").replace("كتب", "").strip() if soup.find("span", attrs={"class": "writeBy"}) else "No Writer"
        tags = soup.find("meta", attrs={"name": "keywords"})["content"] if soup.find("meta", attrs={"name": "keywords"}) else "No Tags"
        article_body = soup.find("div", attrs={"id": "articleBody"}).text if soup.find("div", attrs={"id": "articleBody"}) else "No Content"
        article = " ".join(re.findall(r'[\w\._,"]+', article_body))

        return {
            "url": url,
            "title": title,
            "image": image,
            "writer": writer,
            "tags": tags,
            "article": article,
        }
    except Exception as e:
        print(f"Error fetching article details from {url}: {e}")
        return None

In [5]:
def get_links_from_page(url, page_no):

    try:
        data = requests.get(f"{url}{page_no}").text
        soup = BeautifulSoup(data, "html.parser")
        links = [
            "https://youm7.com" + a["href"]
            for a in soup.find("div", attrs={"id": "paging"}).find_all("a")
        ]
        return list(set(links))
    except Exception as e:
        print(f"Error fetching links from page {url}{page_no}: {e}")
        return []

In [6]:
def get_article_details_by_news_type(news_type_url, number_of_pages):

    articles = []
    for i in range(1, number_of_pages + 1):
       # print(f"Processing page {i} for {news_type_url}")
        links = get_links_from_page(news_type_url, i)
        
        for link in links[:3]:  # Limit to first 5 for testing
            article_details = get_article_details(link)
            if article_details:
                articles.append(article_details)
            time.sleep(1)  
    return pd.DataFrame(articles) , 

In [7]:
def get_all_articles_links(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        links = [
            "https://youm7.com" + h2.find("a")["href"]
            for h2 in soup.find_all("h2") if h2.find("a")
        ]
        return list(set(links))
    except Exception as e:
        print(f"Error fetching all article links: {e}")
        return []

In [9]:
def get_news_type_name(link):
    try:
        response = requests.get(link)
        soup2 =  BeautifulSoup(response.content, 'html.parser')
        news_type_name = soup2.title.text
        return news_type_name
    except:
        return ""

starting the process of extracting data from the web limiting the scope of extraction to only 2 pages with full details

In [10]:
url = "https://www.youm7.com/"
all_articles_links = get_all_articles_links(url)
all_articles_links = all_articles_links[5:7]  # Use only the first 2 for testing


all_df = dict()

for i in range(len(all_articles_links)):
    df = get_article_details_by_news_type(all_articles_links[i], 1)
    key = get_news_type_name(all_articles_links[i])
    all_df[key] = df
       

exploring data new structure of the data and how it can be handled throw the rest of the project

In [11]:
all_articles_links

['https://youm7.com/Section/%D8%A7%D9%82%D8%AA%D8%B5%D8%A7%D8%AF-%D9%88%D8%A8%D9%88%D8%B1%D8%B5%D8%A9/297/1',
 'https://youm7.com/Section/%D8%B5%D8%AD%D8%A9-%D9%88%D8%B7%D8%A8/245/1']

In [12]:
all_df.keys()

dict_keys(['اقتصاد وبورصة - اليوم السابع', 'صحة وطب - اليوم السابع'])

In [13]:
list(all_df.values())[0][0]['article'].tolist()

['قام وفد من المجلس التصديري للحرف اليدوية والصناعات الإبداعية برئاسة السيد هشام العيسوي رئيس المجلس بزيارة معرض "تراثنا" للحرف اليدوية في دورته السادسة المقام في الفترة من 12 ديسمبر حتى 21 ديسمبر الجاري ياتي ذلك في إطار دعمه المستمر للقطاع الحرفي وحرصه على رفع كفاءته. وشارك في الزيارة من أعضاء المجلس التصديري للحرف اليدوية كل من الدكتورة علا حمدي والمصمم محمد سامي والمهندسة تسنيم أيمن عضو امين الصندوق وقد حرص الوفد على تفقد كافة أجنحة العارضين والاستماع إلى آرائهم بشأن الدورة الجديدة للمعرض بالإضافة إلى إجراء مناقشات فعالة تهدف إلى توجيه العارضين بشكل سليم وتحفيزهم على الاهتمام بتوسيع نطاق التصدير. وفي تصريح له قال هشام العيسوي إن زيارة المجلس التصديري للمعرض تأتي في إطار دورنا الأساسي في دعم الحرفيين والمساهمة في رفع قيمة منتجاتهم بما يعزز قدرتها على المنافسة في الأسواق العالمية. وأضاف العيسوي أن المهم أن تتسم هذه المنتجات بالجودة العالية لتكون قابلة للتصدير وهذا ما نعمل على ترسيخه بشكل مستمر." وأضاف العيسوي خلال الجولة "لقد لاحظنا خلال زيارتنا أن العديد من الحرفيين يمتلكون إمكانيات 

In [14]:
list(all_df.keys())

['اقتصاد وبورصة - اليوم السابع', 'صحة وطب - اليوم السابع']

In [15]:
type([x[0] for x in list(all_df.values())][1])

pandas.core.frame.DataFrame

In [16]:
[("\n\n".join(x['article'].tolist())) if len(x)>0 else next  for x in [x[0] for x in list(all_df.values())] ]

['قام وفد من المجلس التصديري للحرف اليدوية والصناعات الإبداعية برئاسة السيد هشام العيسوي رئيس المجلس بزيارة معرض "تراثنا" للحرف اليدوية في دورته السادسة المقام في الفترة من 12 ديسمبر حتى 21 ديسمبر الجاري ياتي ذلك في إطار دعمه المستمر للقطاع الحرفي وحرصه على رفع كفاءته. وشارك في الزيارة من أعضاء المجلس التصديري للحرف اليدوية كل من الدكتورة علا حمدي والمصمم محمد سامي والمهندسة تسنيم أيمن عضو امين الصندوق وقد حرص الوفد على تفقد كافة أجنحة العارضين والاستماع إلى آرائهم بشأن الدورة الجديدة للمعرض بالإضافة إلى إجراء مناقشات فعالة تهدف إلى توجيه العارضين بشكل سليم وتحفيزهم على الاهتمام بتوسيع نطاق التصدير. وفي تصريح له قال هشام العيسوي إن زيارة المجلس التصديري للمعرض تأتي في إطار دورنا الأساسي في دعم الحرفيين والمساهمة في رفع قيمة منتجاتهم بما يعزز قدرتها على المنافسة في الأسواق العالمية. وأضاف العيسوي أن المهم أن تتسم هذه المنتجات بالجودة العالية لتكون قابلة للتصدير وهذا ما نعمل على ترسيخه بشكل مستمر." وأضاف العيسوي خلال الجولة "لقد لاحظنا خلال زيارتنا أن العديد من الحرفيين يمتلكون إمكانيات 

loading the local llm from memory

In [17]:
import openai
from langchain_openai import ChatOpenAI

In [18]:
llm= ChatOpenAI(base_url ='http://localhost:1237/v1',api_key="ieryohanvnao")

In [19]:
import langchain
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import CharacterTextSplitter


import os
import time

summarization prompt to preform map reduce

In [20]:

map_prompt = ChatPromptTemplate.from_messages(
    [("system", "Write a summary of important point in the following text:\\n\\n{context}")]
)
reduce_template = """
The following is a set of summaries:
{docs}
Take these and distill it into a final, consolidated summary
of the main themes.
"""

reduce_prompt = ChatPromptTemplate([("human", reduce_template)])

prepering final summary stucture

In [21]:
summarize={}
k=0

further data investigation 

In [22]:
for x,docs in zip(list(all_df.keys()),[("\n\n".join(x['article'].tolist())) if len(x)>0 else next  for x in [x[0] for x in list(all_df.values())] ] ):
    if not isinstance(docs, str):
        continue
    print(docs)

قام وفد من المجلس التصديري للحرف اليدوية والصناعات الإبداعية برئاسة السيد هشام العيسوي رئيس المجلس بزيارة معرض "تراثنا" للحرف اليدوية في دورته السادسة المقام في الفترة من 12 ديسمبر حتى 21 ديسمبر الجاري ياتي ذلك في إطار دعمه المستمر للقطاع الحرفي وحرصه على رفع كفاءته. وشارك في الزيارة من أعضاء المجلس التصديري للحرف اليدوية كل من الدكتورة علا حمدي والمصمم محمد سامي والمهندسة تسنيم أيمن عضو امين الصندوق وقد حرص الوفد على تفقد كافة أجنحة العارضين والاستماع إلى آرائهم بشأن الدورة الجديدة للمعرض بالإضافة إلى إجراء مناقشات فعالة تهدف إلى توجيه العارضين بشكل سليم وتحفيزهم على الاهتمام بتوسيع نطاق التصدير. وفي تصريح له قال هشام العيسوي إن زيارة المجلس التصديري للمعرض تأتي في إطار دورنا الأساسي في دعم الحرفيين والمساهمة في رفع قيمة منتجاتهم بما يعزز قدرتها على المنافسة في الأسواق العالمية. وأضاف العيسوي أن المهم أن تتسم هذه المنتجات بالجودة العالية لتكون قابلة للتصدير وهذا ما نعمل على ترسيخه بشكل مستمر." وأضاف العيسوي خلال الجولة "لقد لاحظنا خلال زيارتنا أن العديد من الحرفيين يمتلكون إمكانيات را

In [23]:
for x,docs in zip(list(all_df.keys()),["\n\n".join(x['article'].tolist()) for x in [x[0] for x in list(all_df.values())]]):
    print(docs)

قام وفد من المجلس التصديري للحرف اليدوية والصناعات الإبداعية برئاسة السيد هشام العيسوي رئيس المجلس بزيارة معرض "تراثنا" للحرف اليدوية في دورته السادسة المقام في الفترة من 12 ديسمبر حتى 21 ديسمبر الجاري ياتي ذلك في إطار دعمه المستمر للقطاع الحرفي وحرصه على رفع كفاءته. وشارك في الزيارة من أعضاء المجلس التصديري للحرف اليدوية كل من الدكتورة علا حمدي والمصمم محمد سامي والمهندسة تسنيم أيمن عضو امين الصندوق وقد حرص الوفد على تفقد كافة أجنحة العارضين والاستماع إلى آرائهم بشأن الدورة الجديدة للمعرض بالإضافة إلى إجراء مناقشات فعالة تهدف إلى توجيه العارضين بشكل سليم وتحفيزهم على الاهتمام بتوسيع نطاق التصدير. وفي تصريح له قال هشام العيسوي إن زيارة المجلس التصديري للمعرض تأتي في إطار دورنا الأساسي في دعم الحرفيين والمساهمة في رفع قيمة منتجاتهم بما يعزز قدرتها على المنافسة في الأسواق العالمية. وأضاف العيسوي أن المهم أن تتسم هذه المنتجات بالجودة العالية لتكون قابلة للتصدير وهذا ما نعمل على ترسيخه بشكل مستمر." وأضاف العيسوي خلال الجولة "لقد لاحظنا خلال زيارتنا أن العديد من الحرفيين يمتلكون إمكانيات را

loading a pipline for a translation model to get the best out of our model

In [31]:
from deep_translator import GoogleTranslator

def translate_arabic_to_english(text, max_retries=3, delay=1):
    """
    Translate Arabic text to English with error handling
    """
    translator = GoogleTranslator(source='ar', target='en')
    
    for attempt in range(max_retries):
        try:
            if isinstance(text, list):
                return [translator.translate(item) for item in text]
            return translator.translate(text)
        except (requests.exceptions.RequestException, Exception) as e:
            if attempt == max_retries - 1:
                raise Exception(f"Translation failed after {max_retries} attempts: {str(e)}")
            time.sleep(delay)

In [36]:
from transformers import pipeline
pipe = pipeline("text2text-generation", model="facebook/mbart-large-50-many-to-one-mmt")
def translateArabic(arabic_text):
    """
    Translate Arabic text to English
    
    Args:
        arabic_text (str): Input text in Arabic
        
    Returns:
        str: Translated English text
    """
    # Set the source language token for Arabic
    input_text = f"ar_AR: {arabic_text}"
    
    # Generate translation
    try:
        output = pipe(
            input_text,
            forced_bos_token_id=pipe.tokenizer.lang_code_to_id["en_XX"],
            max_length=200000
        )
        
        # Extract translated text
        translated_text = output[0]['generated_text']
        
        # Remove any language tokens that might appear in the output
        translated_text = translated_text.replace('en_XX:', '').strip()
        
        return translated_text
        
    except Exception as e:
        return f"Translation error: {str(e)}"

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All model checkpoint layers were used when initializing TFMBartForConditionalGeneration.

All the layers of TFMBartForConditionalGeneration were initialized from the model checkpoint at facebook/mbart-large-50-many-to-one-mmt.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMBartForConditionalGeneration for predictions without further training.
Device set to use 0


stating the map reduce process to summarize pages

In [37]:
from random import randint

In [58]:
for x,docs in zip(list(all_df.keys()),["\n\n".join(x['article'].tolist()) for x in [x[0] for x in list(all_df.values())]]):
    # if not isinstance(docs, str):
    #     continue;

    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(

        chunk_size=500, chunk_overlap=0
    )
    
    split_docs = text_splitter.split_text(docs)
    # new_splite_docs=[]
    # for dd in split_docs:
    #     new_splite_docs.append(translateArabic(dd))
    #     time.sleep(120)       
    # split_docs =new_splite_docs    
    import operator
    from typing import Annotated, List, Literal, TypedDict

    from langchain.chains.combine_documents.reduce import (
        acollapse_docs,
        split_list_of_docs,
    )
    from langchain_core.documents import Document
    from langgraph.constants import Send
    from langgraph.graph import END, START, StateGraph

    token_max = 1000


    def length_function(documents: List[Document]) -> int:
        """Get number of tokens for input contents."""
        return sum(llm.get_num_tokens(doc.page_content) for doc in documents)


    class OverallState(TypedDict):
        contents: List[str]
        summaries: Annotated[list, operator.add]
        collapsed_summaries: List[Document]
        final_summary: str


    class SummaryState(TypedDict):
        content: str


    async def generate_summary(state: SummaryState):
        prompt = map_prompt.invoke(state["content"])
        response = await llm.ainvoke(prompt)
        return {"summaries": [response.content]}


    def map_summaries(state: OverallState):
        return [
            Send("generate_summary", {"content": content}) for content in state["contents"]
        ]


    def collect_summaries(state: OverallState):
        return {
            "collapsed_summaries": [Document(summary) for summary in state["summaries"]]
        }


    async def _reduce(input: dict) -> str:
        prompt = reduce_prompt.invoke(input)
        response = await llm.ainvoke(prompt)

        return response.content


    async def collapse_summaries(state: OverallState):
        doc_lists = split_list_of_docs(
            state["collapsed_summaries"], length_function, token_max
        )
        results = []
        for doc_list in doc_lists:
            results.append(await acollapse_docs(doc_list, _reduce))

        return {"collapsed_summaries": results}


    def should_collapse(
        state: OverallState,
    ) -> Literal["collapse_summaries", "generate_final_summary"]:
        num_tokens = length_function(state["collapsed_summaries"])
        if num_tokens > token_max:
            return "collapse_summaries"
        else:
            return "generate_final_summary"


    async def generate_final_summary(state: OverallState):
        response = await _reduce(state["collapsed_summaries"])
        return {"final_summary": response}


    graph = StateGraph(OverallState)
    graph.add_node("generate_summary", generate_summary)  # same as before
    graph.add_node("collect_summaries", collect_summaries)
    graph.add_node("collapse_summaries", collapse_summaries)
    graph.add_node("generate_final_summary", generate_final_summary)

    graph.add_conditional_edges(START, map_summaries, ["generate_summary"])
    graph.add_edge("generate_summary", "collect_summaries")
    graph.add_conditional_edges("collect_summaries", should_collapse)
    graph.add_conditional_edges("collapse_summaries", should_collapse)
    graph.add_edge("generate_final_summary", END)

    app = graph.compile()

    async for step in app.astream(
        {"contents": [doc for doc in split_docs]},
        {"recursion_limit": 10},
    ):
        print(list(step.keys()))

    summarize[x] =step['generate_final_summary']['final_summary']

Created a chunk of size 1870, which is longer than the specified 500
Created a chunk of size 3970, which is longer than the specified 500


['generate_summary']
['generate_summary']
['generate_summary']
['collect_summaries']


Created a chunk of size 2507, which is longer than the specified 500
Created a chunk of size 2898, which is longer than the specified 500


['generate_final_summary']
['generate_summary']
['generate_summary']
['generate_summary']
['collect_summaries']
['collapse_summaries']
['generate_final_summary']


final summary preduced 

In [59]:
print(summarize)

{'اقتصاد وبورصة - اليوم السابع': ' A delegation from the Export Council of the Handicrafts and Creative Industries visited the "Heritage" handicraft exhibition. The industries are well-known but mostly confined to domestic markets. The delegates highlighted that craftspeople have immense potential in manufacturing and there are numerous high-quality products that just need proper guidance. \n\nEgypt and Italy are engaged in negotiations to facilitate logistics procedures, aiming to ease import and export processes as the RoRo system is actively implemented. Streamlining logistical procedures between Egypt and Italy will increase the competitive advantage of Egyptian products in the market by reducing production costs. The RoRo system directly links the Port of Alexandria in Egypt to the Port of Trieste in northern Italy, enhancing the efficiency of Egyptian exports to European markets. This creates a regional logistical hub that facilitates movement of goods between Egypt and other Eur

In [81]:
import sqlite3
con = sqlite3.connect('NewsOfElyoum7.db')


In [82]:

cur_sor = con.cursor()
cur_sor.execute('CREATE TABLE news (id INTEGER primary key,summary_id integer, tittle TEXT, url TEXT,kind TEXT,writer TEXT, tags TEXT, article TEXT)')


con.commit()


In [83]:
cur_sor.execute('CREATE TABLE summary (id INTEGER primary key ,kind text,date_time text,suummary text )')
from datetime import datetime

timestamp = int(datetime.now().timestamp() * 1000)

con.commit()


In [84]:
import datetime

# con = sqlite3.connect('news_of_Elyoum7.db',timeout=100)
# cur_sor = con.cursor()

# news_of_youm7 = [
#     (1, 'وزير الخارجية يشارك فى الاجتماع الوزارى الخاص ...', 'https://youm7.com/story/2024/12/7', 'سياسة', 'أحمد جمعة', 'بدر عبد العاطى, وزير الخارجية, سوريا, الدوحة', 'شارك الدكتور بدر عبد العاطي وزير الخارجية ...'),
#     (2, 'وزارة العمل تتلقى غدا طلبات التقدم لفرص بالسعودية...', 'https://youm7.com/story/2024/12/7', 'فرص عمل', 'أية دعبس', 'وزارة العمل, فرص عمل بالخارج, فرص عمل بالسعودية', 'أعلنت وزارة العمل فتح باب التقديم على فرص عمل ...'),
#     (3, 'القاهرة الإخبارية: دخول فصائل مسلحة مناطق في ر...', 'https://youm7.com/story/2024/12/8/', 'سياسة', 'أحمد علوي', 'دمشق, العاصمة السورية, سوريا, بشار الاسد, اخبار سوريا', 'أفادت قناة القاهرة الإخبارية فى نبأ عاجل لها ...')
# ]

bigT=1
smallT=1

for  key in list(all_df.keys()):
    cur_sor.executemany("""
        INSERT INTO summary  
        VALUES (?, ?, ?, ?)
        """, [
            (bigT,key,datetime.datetime.now(),summarize[key])
        ])
    con.commit()
    bigT=bigT+1
    for index, row in all_df[key][0].iterrows():
        cur_sor.executemany("""
            INSERT INTO news  
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            """, [
                (smallT,bigT,row['title'],row['url'],key,row['writer'],row['tags'],row['article'])
            ])
        con.commit()
        smallT= smallT+1




